# Merge Kinetics Libraries
This script is used to merge different kinetics libraries generated from ARC

## 1. Necessary packages

In [ ]:

import os
import sys

from rmgpy.data.kinetics import KineticsDatabase, KineticsLibrary

sys.path.append('/Users/xiaorui/Apps/easy_rmg_model')
from easy_rmg_model.rmg2arc.kinetics_db import (find_kinetics_libs,
                                                load_kinetics_database,
                                                load_kinetics_lib_by_path,
                                                merge_kinetics_lib)

%load_ext autoreload
%autoreload 2
%matplotlib inline

## 2. Input
Merge the libraries from working directory and save the new library into the save path. If the save path is orignially a library, then merge everything on this library.
- `save_path` (str): the **full path (to folder) to save the merged library**. If the path exists, the file will be overwriten.
- `work_dir` (str): indicate **where your results are**. The script will search and find all of the kinetics libraries by identifying `reactions.py` that has `dictionary.txt` under the same folderm. 
- `complementary_ARC_project_paths` (list): a list of **complementary ARC projects paths** which is not in the `work_dir`
- `complementary_lib_paths` (list): a list of **complementary thermo library file paths** to be merged, so that you can also merge files that are not generated by ARC
- `rmg_builtin_libraries` (list): a list of thermo **library names**. These are libraries placed in `RMG-database/input/thermo/libraries/`m

In [ ]:
save_path = '/Volumes/Extreme SSD/relax-rotor/TS/Intra addition/'
work_dir = '/Volumes/Extreme SSD/relax-rotor/TS/Intra addition/'
complementary_ARC_project_paths = []
complementary_lib_paths = []
rmg_builtin_libraries = []

## 3. Load a RMG database instance

In [ ]:
kinetics_db = load_kinetics_database(families='default', libraries=rmg_builtin_libraries)

In [ ]:
kinetics_db = load_kinetics_database(families='none', libraries=rmg_builtin_libraries)
lib_list = find_kinetics_libs(work_dir) + complementary_lib_paths

for arc_project in complementary_ARC_project_paths:
    lib_list += find_kinetics_libs(arc_project)

for lib in lib_list:
    try:
        load_kinetics_lib_by_path(path=lib,
                                  kinetics_db=kinetics_db,
                                  reload=False)
    except TypeError:
        # Problematic library
        print(f'Find a problematic library: {lib}')
        raise


lib_list = lib_list + rmg_builtin_libraries

## 4. Initalize the library
Initalize the library to be saved and library contains thermo to be reconsidered

In [ ]:
base_lib = KineticsLibrary(label='The merging library',
                           name='The merging library')

# Also create a library to store kinetics if there 
# is a conflict and you haven't decided whether to add it
tbd_lib = KineticsLibrary(label='TBD kinetics',
                         name='TBD kinetics',)

## 5. Combine the thermo libraries 

In [ ]:
for lib_label in lib_list:
    try:
        library_to_add = kinetics_db.libraries[lib_label]
    except KeyError:
        continue
    merge_kinetics_lib(base_lib, library_to_add, tbd_lib)

## 6. Save the libraries

In [ ]:
base_lib.save(os.path.join(save_path, 'reactions.py'))

os.makedirs(os.path.join(save_path, 'tbd'), exist_ok=True)
tbd_lib.save(os.path.join(save_path, 'tbd', 'reactions.py'))
print(f"Totally {len(base_lib.entries)} reactions in the merged library.")

# Test if the merged database can be loaded
try:
    lib = KineticsLibrary()
    lib.load(os.path.join(save_path, 'reactions.py'),
             KineticsDatabase().local_context,
             KineticsDatabase().global_context)
except:
    print('The library can not be loaded. You need to manually check it.')
else:
    print('The merged library is now ready to use.')
    